<a href="https://colab.research.google.com/github/FMurray/hyperdemocracy/blob/main/hyper_democracy_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [1]:
# if you are on a google colab, uncomment the lines below to fetch the requirements file and the hyperdemocracy.py module
# and pip install the requirements

#!wget https://raw.githubusercontent.com/FMurray/hyperdemocracy/main/requirements.txt
#!wget https://raw.githubusercontent.com/FMurray/hyperdemocracy/main/hyperdemocracy.py
!pip install -r requirements.txt

In [2]:
import os
import rich
import time
import tiktoken
from tqdm import tqdm
from transformers import AutoTokenizer
from rich import print

# Choose a Provider

We have options for HuggingFace and OpenAI model providers in this notebook.

In [3]:
PROVIDER = "HF"  
assert PROVIDER in ["HF", "OPENAI"]

CONFIGS = {
    "HF": {
        "embd": "sentence-transformers/all-mpnet-base-v2",
        "llm": "google/flan-t5-large",
    },
    "OPENAI": {
        "embd": "text-embedding-ada-002",
        "llm": "gpt-3.5-turbo-16k",
    },
}

CONFIG = CONFIGS[PROVIDER]

# Setup Keys

In [4]:
# if you want to use local secrets, add a file called .env to this directory and uncomment the lines below

from dotenv import load_dotenv
load_dotenv(".env")

True

In [5]:
# if you are using google colab, uncomment the lines below to manually enter your OpenAI key.

#import getpass
#os.environ['OPENAI_API_KEY'] = getpass.getpass()

In [6]:
# if you are using google colab, uncomment the lines below to manually enter your HuggingFace token.

#import getpass
#os.environ['HUGGINGFACEHUB_API_TOKEN'] = getpass.getpass()

In [7]:
# this is for development
#%load_ext autoreload
#%autoreload 2

# Load Assembled Records

We are going to use a small subset of records provided by https://assembled.app/.

For the purposes of this workshop, we have created a [huggingface dataset](https://huggingface.co/datasets/assembleco/hyperdemocracy)  which we can load using the `load_dataset` function. This is all handled for you in the `load_assembleco_records` function. See more info here [datasets](https://huggingface.co/docs/datasets/index) package.

In [8]:
from hyperdemocracy import load_assembleco_records

In [9]:
df = load_assembleco_records(process=True, strip_html=True, remove_empty_body=True)

Found cached dataset parquet (/home/calliope/.cache/huggingface/datasets/assembleco___parquet/assembleco--hyperdemocracy-a598a9b2b17e51dc/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [10]:
df.head()

,key,name,sponsors,summary,body,themes,index,actions,amendments,committees,relatedbills,cosponsors,subjects,text,titles,congress_num,legis_class,legis_num,congress_gov_url
0,118HCONRES1,Regarding consent to assemble outside the seat...,"[[C001053, Rep. Cole, Tom [R-OK-4], sponsor]]",This concurrent resolution authorizes the Spe...,[Congressional Bills 118th Congress]\n[From th...,"[Congress, Congressional operations and organi...","{'bill': {'actions': {'count': 7, 'url': 'http...","{'actions': [{'actionCode': None, 'actionDate'...","{'amendments': [], 'pagination': {'count': 0},...","{'committees': [], 'request': {'billNumber': '...","{'pagination': {'count': 0}, 'relatedBills': [...","{'cosponsors': [], 'pagination': {'count': 0, ...","{'pagination': {'count': 2}, 'request': {'bill...","{'pagination': {'count': 1}, 'request': {'bill...","{'pagination': {'count': 2}, 'request': {'bill...",118,HCONRES,1,https://www.congress.gov/bill/118th-congress/h...
1,118HCONRES10,Expressing the sense of Congress that the Unit...,"[[T000165, Rep. Tiffany, Thomas P. [R-WI-7], s...",This concurrent resolution calls on the Presi...,[Congressional Bills 118th Congress]\n[From th...,[International Affairs],"{'bill': {'actions': {'count': 4, 'url': 'http...","{'actions': [{'actionCode': 'H11100', 'actionD...","{'amendments': [], 'pagination': {'count': 0},...",{'committees': [{'activities': [{'date': '2023...,"{'pagination': {'count': 0}, 'relatedBills': [...","{'cosponsors': [{'bioguideId': 'P000605', 'dis...","{'pagination': {'count': 1}, 'request': {'bill...","{'pagination': {'count': 1}, 'request': {'bill...","{'pagination': {'count': 2}, 'request': {'bill...",118,HCONRES,10,https://www.congress.gov/bill/118th-congress/h...
2,118HCONRES11,Providing for a joint session of Congress to r...,"[[S001176, Rep. Scalise, Steve [R-LA-1], spons...",This concurrent resolution provides for a joi...,[Congressional Bills 118th Congress]\n[From th...,"[Congress, Congressional operations and organi...","{'bill': {'actions': {'count': 10, 'url': 'htt...","{'actions': [{'actionCode': None, 'actionDate'...","{'amendments': [], 'pagination': {'count': 0},...","{'committees': [], 'request': {'billNumber': '...","{'pagination': {'count': 0}, 'relatedBills': [...","{'cosponsors': [], 'pagination': {'count': 0, ...","{'pagination': {'count': 3}, 'request': {'bill...","{'pagination': {'count': 3}, 'request': {'bill...","{'pagination': {'count': 2}, 'request': {'bill...",118,HCONRES,11,https://www.congress.gov/bill/118th-congress/h...
3,118HCONRES12,Expressing the sense of Congress that all dire...,"[[C001039, Rep. Cammack, Kat [R-FL-3], sponsor...",This concurrent resolution expresses the sens...,[Congressional Bills 118th Congress]\n[From th...,"[Foreign Trade and International Finance, Agri...","{'bill': {'actions': {'count': 5, 'url': 'http...","{'actions': [{'actionCode': 'H11000', 'actionD...","{'amendments': [], 'pagination': {'count': 0},...",{'committees': [{'activities': [{'date': '2023...,"{'pagination': {'count': 0}, 'relatedBills': [...","{'cosponsors': [{'bioguideId': 'K000380', 'dis...","{'pagination': {'count': 6}, 'request': {'bill...","{'pagination': {'count': 1}, 'request': {'bill...","{'pagination': {'count': 2}, 'request': {'bill...",118,HCONRES,12,https://www.congress.gov/bill/118th-congress/h...
4,118HCONRES13,Supporting the Local Radio Freedom Act.,"[[W000809, Rep. Womack, Steve [R-AR-3], sponso...",This concurrent resolution declares that Cong...,[Congressional Bills 118th Congress]\n[From th...,"[Science, Technology, Communications, Congress]","{'bill': {'actions': {'count': 3, 'url': 'http...","{'actions': [{'actionCode': 'H11100', 'actionD...","{'amendments': [], 'pagination': {'count': 0},...",{'committees': [{'activities': [{'date': '2023...,"{'pagination': {'count': 1}, 'relatedBills': [...","{'cosponsors': [{'bioguideId': 'C001066', 'dis...","{'pagination': {'count': 2}, 'request': {'bill...","{'pagination': {'count': 1}, 'request': {'b

In [11]:
print(len(df))

6132

In [12]:
df.shape

(6132, 19)

# Sponsor Graph Sidequest

We will be focusing on the text content of the legislation in this workshop, but if you would like to explore building a graph from the sponsor / co-sponsor / legislation network check out the [sponsor_graph notebook](https://github.com/FMurray/hyperdemocracy/blob/main/sidequests/sponsor_graph.ipynb) to get started.

# From Pandas Dataframe to LangChain Documents

A langchain document is a simple class with two attributes, 
* page_content (a string)
* metadata (a dictionary)

In [13]:
from langchain.schema import Document 

In [14]:
Document??

Below we take each row from our legislation DataFrame and create a LangChain Document. We use the `body` column for the `page_content` attribute and populate the `metadata` attribute with data from some of the other columns. Note that the `source` key in the `metadata` dictionary is associated with a congress.gov url. The `source` key can hold an arbitrary string and will become important when we look into question answering systems that return information about the sources used to answer a question. We also restrict ourselves to `str`, `int`, and `float` types in the other values of our `metadata` dictionary. This is to make it easy to use them as filters when querying our vectorstore. If that doesn't make sense, dont worry! It will by the end of the workshop.  

In [15]:
all_docs = []
for irow, row in df.iterrows():
    doc = Document(
        page_content=row['body'],
        metadata={
            # Note: chroma can only filter on float, str, or int
            # https://docs.trychroma.com/usage-guide#using-where-filters
            'key': row['key'],
            'congress_num': row['congress_num'],
            'legis_class': row['legis_class'],
            'legis_num': row['legis_num'],
            'name': row['name'],
            'summary': row['summary'],
            'sponsor': row['sponsors'][0][0],
            'source': row['congress_gov_url'],
        },
    )
    all_docs.append(doc)

In [16]:
print(all_docs[0])

Document(
    page_content='[Congressional Bills 118th Congress]\n[From the U.S. Government Publishing Office]\n[H. Con. Res.
1 Engrossed in House (EH)]\n\n<DOC>\n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  
\n  \n  \n  \n118th CONGRESS\n  1st Session\nH. CON. RES. 
1\n\n_______________________________________________________________________\n\n                         CONCURRENT
RESOLUTION\n\n    Resolved by the House of Representatives (the Senate concurring), \nThat pursuant to clause 4, 
section 5, article I of the Constitution, \nduring the One Hundred Eighteenth Congress the Speaker of the House and
\nthe Majority Leader of the Senate or their respective designees, acting \njointly after consultation with the 
Minority Leader of the House and \nthe Minority Leader of the Senate,\n\n\n              \n\n may notify the 
Members of the House and the Senate, respectively, to \nassemble at a place outside the District of Columbia if, in
their \nopinion, the public interest shall warrant it.\n\n            Passed the House of Representatives January 
9, 2023.\n\n            Attest:\n\n                                                                 Clerk.\n118th 
CONGRESS\n\n  1st Session\n\n                             H. CON. RES. 
1\n\n_______________________________________________________________________\n\n                         CONCURRENT
RESOLUTION\n\n     Regarding consent to assemble outside the seat of government.\n\n',
    metadata={
        'key': '118HCONRES1',
        'congress_num': '118',
        'legis_class': 'HCONRES',
        'legis_num': '1',
        'name': 'Regarding consent to assemble outside the seat of government.',
        'summary': ' This concurrent resolution authorizes the Speaker of the House and the Majority Leader of the 
Senate to assemble the House and the Senate outside the District of Columbia whenever the public interest warrants 
it.',
        'sponsor': 'C001053',
        'source': 'https://www.congress.gov/bill/118th-congress/house-concurrent-resolution/1'
    }
)

## Activity

* examine the Document content
* visit the congress.gov URL and view the document in various formats
* examine the body text below
* read the summary of the document and attempt to connect it with the long form text of the document

In [17]:
print(all_docs[0].page_content)

[Congressional Bills 118th Congress]
[From the U.S. Government Publishing Office]
[H. Con. Res. 1 Engrossed in House (EH)]

<DOC>
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
118th CONGRESS
  1st Session
H. CON. RES. 1

_______________________________________________________________________

                         CONCURRENT RESOLUTION

    Resolved by the House of Representatives (the Senate concurring), 
That pursuant to clause 4, section 5, article I of the Constitution, 
during the One Hundred Eighteenth Congress the Speaker of the House and 
the Majority Leader of the Senate or their respective designees, acting 
jointly after consultation with the Minority Leader of the House and 
the Minority Leader of the Senate,


              

 may notify the Members of the House and the Senate, respectively, to 
assemble at a place outside the District of Columbia if, in their 
opinion, the public interest shall warrant it.

            Passed the House of Representatives January 9, 2023.

            Attest:

                                                                 Clerk.
118th CONGRESS

  1st Session

                             H. CON. RES. 1

_______________________________________________________________________

                         CONCURRENT RESOLUTION

     Regarding consent to assemble outside the seat of government.

In [18]:
print(len(all_docs))

6132

# Subsample Docs

In [19]:
NUM_DOCS = 6132
# NUM_DOCS = 50
docs = all_docs[:NUM_DOCS]

In [20]:
from langchain.callbacks import get_openai_callback

In [21]:
def count_openai_tokens_in_docs(docs, model_name=CONFIG["embd"]):
    num_tokens = 0
    enc = tiktoken.encoding_for_model(model_name)
    for doc in docs:
        num_tokens += len(enc.encode(doc.page_content))
    return num_tokens

In [22]:
def count_hf_tokens_in_docs(docs, model_name=CONFIG["embd"]):
    num_tokens = 0
    tokenizer = AutoTokenizer.from_pretrained(CONFIG["embd"])
    for doc in docs:
        num_tokens += len(tokenizer(docs[0].page_content)['input_ids'])
    return num_tokens

In [23]:
# estimate cost
if PROVIDER == "OPENAI":
    num_tokens = count_openai_tokens_in_docs(docs)
    cost = EMBED_DOLLARS_PER_1K_TOKENS * num_tokens / 1000
    print('Num Docs: ', len(docs))
    print('Num Tokens: ', num_tokens)
    print('Total Cost (USD): ', '$'+str(cost))
elif PROVIDER == "HF":
    num_tokens = count_hf_tokens_in_docs(docs)
    cost = 0
    print('Num Docs: ', len(docs))
    print('Num Tokens: ', num_tokens)
    print('Total Cost (USD): ', '$'+str(cost))

Num Docs:  6132

Num Tokens:  2115540

Total Cost (USD):  $0

## Activity

* Contemplate why the answers are slightly different between the "QA" result and the "QA with sources" result.
* Visit the source links and check if the linked legislation is relevant to the question.

# Document QA - Step by Step

Our goal is to setup a question answering (QA) system that can repond to natural language questions about legislation using source material that we provide. In the following sections we will unpack all of components and go over them in detail.

# Part 1 - Langchain Text Splitters

> When you want to deal with long pieces of text, it is necessary to split up that text into chunks. As simple as this sounds, there is a lot of potential complexity here. Ideally, you want to keep the semantically related pieces of text together. What "semantically related" means could depend on the type of text. This notebook showcases several ways to do that.

> At a high level, text splitters work as following:

>    1. Split the text up into small, semantically meaningful chunks (often sentences).
>    2. Start combining these small chunks into a larger chunk until you reach a certain size (as measured by some function).
>    3. Once you reach that size, make that chunk its own piece of text and then start creating a new chunk of text with some overlap (to keep context between chunks).

> That means there are two different axes along which you can customize your text splitter:

>    1. How the text is split
>    2. How the chunk size is measured

-- https://python.langchain.com/docs/modules/data_connection/document_transformers/#text-splitters

Here are some useful options for splitting legislative text, 

* [character text splitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/character_text_splitter)
  * How the text is split: by single character
  * How the chunk size is measured: by number of characters
* [recursive text splitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter)
  * How the text is split: by list of characters
  * How the chunk size is measured: by number of characters
* [split by token](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/split_by_token)
  * How the text is split: by character passed in
  * How the chunk size is measured: by tiktoken tokenizer

If you are not familiar with the concept of a token, this article may help, 
* https://simonwillison.net/2023/Jun/8/gpt-tokenizers/

Mini Side Quest
* see if there is anything interesting that can be done with this https://twitter.com/RLanceMartin/status/1670489431168659456?s=20

In [24]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import TokenTextSplitter
from langchain.text_splitter import SentenceTransformersTokenTextSplitter

In [25]:
text = """We hold these truths to be self-evident, that all men are created equal,

that they are endowed by their Creator with certain unalienable Rights,

that among these are Life, Liberty and the pursuit of Happiness."""

## CharacterTextSplitter

In [26]:
# this is the default separator
CharacterTextSplitter(separator="\n\n", chunk_size=20, chunk_overlap=0).split_text(text)

Created a chunk of size 72, which is longer than the specified 20
Created a chunk of size 71, which is longer than the specified 20


['We hold these truths to be self-evident, that all men are created equal,',
 'that they are endowed by their Creator with certain unalienable Rights,',
 'that among these are Life, Liberty and the pursuit of Happiness.']

In [27]:
# this is what happens if we chandetailge the default separator
CharacterTextSplitter(separator=" ", chunk_size=20, chunk_overlap=0).split_text(text)

['We hold these truths',
 'to be self-evident,',
 'that all men are',
 'created equal,\n\nthat',
 'they are endowed by',
 'their Creator with',
 'certain unalienable',
 'Rights,\n\nthat among',
 'these are Life,',
 'Liberty and the',
 'pursuit of',
 'Happiness.']

In [28]:
# this is what overlap does
CharacterTextSplitter(separator=" ", chunk_size=20, chunk_overlap=10).split_text(text)

['We hold these truths',
 'truths to be',
 'to be self-evident,',
 'that all men are',
 'men are created',
 'created equal,\n\nthat',
 'they are endowed by',
 'endowed by their',
 'by their Creator',
 'Creator with certain',
 'certain unalienable',
 'Rights,\n\nthat among',
 'among these are',
 'these are Life,',
 'are Life, Liberty',
 'Liberty and the',
 'and the pursuit of',
 'of Happiness.']

## RecursiveCharacterTextSplitter

In [29]:
# these are the default separators
RecursiveCharacterTextSplitter(separators=["\n\n", "\n", " ", ""], chunk_size=40, chunk_overlap=0).split_text(text)

['We hold these truths to be self-evident,',
 'that all men are created equal,',
 'that they are endowed by their Creator',
 'with certain unalienable Rights,',
 'that among these are Life, Liberty and',
 'the pursuit of Happiness.']

In [30]:
# this is what happens if we add "," to the separators
RecursiveCharacterTextSplitter(separators=["\n\n", "\n", ",", " ", ""], chunk_size=40, chunk_overlap=0).split_text(text)

['We hold these truths to be self-evident',
 ', that all men are created equal,',
 'that they are endowed by their Creator',
 'with certain unalienable Rights',
 ',',
 'that among these are Life',
 ', Liberty and the pursuit of Happiness.']

## TokenTextSplitter

In [31]:
# the length unit for chunk_size is now tokens not characters
if PROVIDER == "HF":
    ts = SentenceTransformersTokenTextSplitter(
        model_name=CONFIG["embd"],
        chunk_size=10, 
        tokens_per_chunk=10,
        chunk_overlap=0,
    )
elif PROVIDER == "OPENAI":
    ts = TokenTextSplitter(
        model_name=CONFIG["embd"], 
        chunk_size=10, 
        chunk_overlap=0,
    )

In [32]:
ts.split_text(text)

['we hold these truths to be self - evident,',
 'that all men are created equal, that they are',
 'endowed by their creator with certain unalienable rights',
 ', that among these are life, liberty and the',
 'pursuit of happiness.']

In [33]:
# same for chunk_overlap
if PROVIDER == "HF":
    ts = SentenceTransformersTokenTextSplitter(
        model_name=CONFIG["embd"],
        chunk_size=10, 
        tokens_per_chunk=10,
        chunk_overlap=4,
    )
elif PROVIDER == "OPENAI":
    ts = TokenTextSplitter(
        model_name=CONFIG["embd"], 
        chunk_size=10, 
        chunk_overlap=4,
    )
ts.split_text(text)

['we hold these truths to be self - evident,',
 'self - evident, that all men are created equal',
 'men are created equal, that they are endowed by',
 'they are endowed by their creator with certain unalie',
 'with certain unalienable rights, that among these',
 ', that among these are life, liberty and the',
 ', liberty and the pursuit of happiness.',
 'happiness.']

## Lets Make a TextSplitter Choice here

In [34]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=128)
split_docs = text_splitter.split_documents(docs)

In [35]:
print("Number of original docs: ", len(docs))
print("Number of split docs: ", len(split_docs))

Number of original docs:  6132

Number of split docs:  183680

In [36]:
print(split_docs[50])

Document(
    page_content='local radio for news, weather, and entertainment; and such a performance \n        fee is not 
justified when the current system has produced the most \n        prolific and innovative broadcasting, music, and 
sound recording \n        industries in the world: Now, therefore, be it\n    Resolved by the House of 
Representatives (the Senate concurring), \nThat Congress should not impose any new performance fee, tax, royalty, 
\nor other charge relating to the public performance of sound recordings',
    metadata={
        'key': '118HCONRES13',
        'congress_num': '118',
        'legis_class': 'HCONRES',
        'legis_num': '13',
        'name': 'Supporting the Local Radio Freedom Act.',
        'summary': ' This concurrent resolution declares that Congress should not impose any new performance fee, 
tax, royalty, or other charge relating to the public performance of sound recordings on a local radio station for 
broadcasting sound recordings over the air, or on any business for such public performance of sound recordings.',
        'sponsor': 'W000809',
        'source': 'https://www.congress.gov/bill/118th-congress/house-concurrent-resolution/13'
    }
)

# Part 2 - Embed and Index Doc Chunks

Now we will embed and index the document chunks from the previous section. 
We have many choices when it comes to text embedding models and vector indexes. 
For this tutorial we will choose, 

* text embedding model: `text_embedding_ada_002`
* vector index:
  * https://www.trychroma.com
  * https://docs.trychroma.com/usage-guide#changing-the-distance-function
  * https://github.com/nmslib/hnswlib/tree/master#supported-distances
  * https://github.com/hwchase17/langchain/blob/master/langchain/vectorstores/chroma.py
  * https://github.com/hwchase17/langchain/blob/master/langchain/vectorstores/utils.py#L10

For a look at some of the top performing closed and open source text embedding models, check out the HuggingFace Massive Text Embedding Benchmark (MTEB), 
* https://huggingface.co/spaces/mteb/leaderboard
  
For a more detailed introduction to embeddings in general, see the embeddings notebook
* https://github.com/FMurray/hyperdemocracy/blob/main/sidequests/embeddings_v2.ipynb

In [37]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

In [38]:
if PROVIDER == "HF":
    embeddings = HuggingFaceEmbeddings(model_name=CONFIG["embd"])
elif PROVIDER == "OPENAI":
    embeddings = OpenAIEmbeddings(model=CONFIG["embd"])

In [39]:
persist_directory = f"hyperdemocracy-chromadb-prov-{PROVIDER}-ndocs-{NUM_DOCS}"
print(persist_directory)

hyperdemocracy-chromadb-prov-HF-ndocs-6132

In [40]:
## THIS CELL SPENDS MONEY THE FIRST TIME ##
if os.path.exists(persist_directory):
    vec_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
else:
    batch_size = 128
    for ii in tqdm(range(0, len(split_docs), batch_size)):
        batch = split_docs[ii:ii+batch_size]
        if ii == 0:
            vec_store = Chroma.from_documents(batch, embeddings, persist_directory=persist_directory)
        else:
            vec_store.add_documents(batch)
        time.sleep(1.0)
    vec_store.persist()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [41]:
vec_store

In [42]:
ret_docs = vec_store.similarity_search_with_score(
    "commercial relations in Europe", 
    k=3, 
    filter={"source": "https://www.congress.gov/bill/118th-congress/house-concurrent-resolution/17"},
)

print("number of returned docs: ", len(ret_docs))
for doc in ret_docs:
    print(doc)

number of returned docs:  3

(
    Document(
        page_content='producer in the world;\nWhereas, in 2019, the United States became a net exporter of 
petroleum products \n        for the first time since 1952, and the reliance of the United States on \n        
foreign imports of petroleum products has declined to historic lows; and\nWhereas free trade, open markets, and 
competition have contributed to the rise \n        of the United States as a global energy superpower: Now, 
therefore, be \n        it',
        metadata={
            'key': '118HCONRES17',
            'congress_num': '118',
            'legis_class': 'HCONRES',
            'legis_num': '17',
            'name': 'Expressing the sense of Congress that the Federal Government should not impose any 
restrictions on the export of crude oil or other petroleum products.',
            'summary': '',
            'sponsor': 'G000558',
            'source': 'https://www.congress.gov/bill/118th-congress/house-concurrent-resolution/17'
        }
    ),
    1.3940151929855347
)

(
    Document(
        page_content='of the United States as a global energy superpower: Now, therefore, be \n        it\n    
Resolved by the House of Representatives (the Senate concurring), \nThat it is the sense of Congress that the 
Federal Government should not \nimpose--\n            (1) overly restrictive regulations on the exploration, \n    
production, or marketing of energy resources; or\n            (2) any restrictions on the export of crude oil or 
other \n        petroleum products under the Energy Policy and Conservation Act',
        metadata={
            'key': '118HCONRES17',
            'congress_num': '118',
            'legis_class': 'HCONRES',
            'legis_num': '17',
            'name': 'Expressing the sense of Congress that the Federal Government should not impose any 
restrictions on the export of crude oil or other petroleum products.',
            'summary': '',
            'sponsor': 'G000558',
            'source': 'https://www.congress.gov/bill/118th-congress/house-concurrent-resolution/17'
        }
    ),
    1.4783275127410889
)

(
    Document(
        page_content='Whereas the United States has enjoyed a renaissance in energy production, with \n        the 
expansion of domestic crude oil and other petroleum product \n        production contributing to enhanced energy 
security and significant \n        economic benefits to the national economy;\nWhereas, in 2015, Congress 
recognized the need to adapt to changing crude oil \n        market conditions and repealed all restrictions on the
export of crude \n        oil on a bipartisan basis;',
        metadata={
            'key': '118HCONRES17',
            'congress_num': '118',
            'legis_class': 'HCONRES',
            'legis_num': '17',
            'name': 'Expressing the sense of Congress that the Federal Government should not impose any 
restrictions on the export of crude oil or other petroleum products.',
            'summary': '',
            'sponsor': 'G000558',
            'source': 'https://www.congress.gov/bill/118th-congress/house-concurrent-resolution/17'
        }
    ),
    1.5029969215393066
)

# Part 3 - Build A RetrievalQA Chain

In [43]:
import langchain
langchain.verbose = False

In [44]:
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.llms import HuggingFaceHub
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

# base classes to examine
from langchain.vectorstores.base import VectorStore
from langchain.schema import BaseRetriever

## Create a Retriever from Chroma VectorStore

In [45]:
retriever = vec_store.as_retriever(search_kwargs={'k':5})

In [46]:
vec_store

In [47]:
retriever

VectorStoreRetriever(vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x7f6aa33e04d0>, search_type='similarity', search_kwargs={'k': 5})

## Plug in a Language Model

With LangChain we can use a text completion model or a chat model for QA.

In [48]:
if PROVIDER == "HF":
    llm = HuggingFaceHub(
        repo_id=CONFIG["llm"], 
        model_kwargs={
            "temperature": 0,
            "max_length": 512,
        })
elif PROVIDER == "OPENAI":
    if CONFIG["llm"].startswith("text"):
        llm = OpenAI(model_name=CONFIG["llm"], temperature=0)
    elif CONFIG["llm"].startswith("gpt"):
        llm = ChatOpenAI(model_name=CONFIG["llm"], temperature=0)

In [49]:
llm

HuggingFaceHub(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, client=InferenceAPI(api_url='https://api-inference.huggingface.co/pipeline/text2text-generation/google/flan-t5-large', task='text2text-generation', options={'wait_for_model': True, 'use_gpu': False}), repo_id='google/flan-t5-large', task=None, model_kwargs={'temperature': 0, 'max_length': 512}, huggingfacehub_api_token=None)

In [50]:
# create a RetrievalQA Chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True,
)

In [51]:
response = qa("How can China be encouraged to discuss land deals in middle Asia?")
print(response['result'])
print(response['source_documents'])

regular, transparent, and unmanipulated visits by members of the press, Members of Congress, congressional staff 
delegations, the United States Special Coordinator for Uyghur Issues under section 4

[
    Document(
        page_content="Congress finds the following:\n            (1) It has been the long-standing policy of the 
United \n        States to encourage meaningful and direct dialogue between \n        People's Republic of China 
authorities and the Dalai Lama or \n        his representatives, without preconditions, to seek a \n        
settlement that resolves differences.\n            (2) Ten rounds of dialogue held between 2002 and 2010 \n        
between the People's Republic of China authorities and the 14th",
        metadata={
            'key': '118HR533',
            'congress_num': '118',
            'legis_class': 'HR',
            'legis_num': '533',
            'name': 'Promoting a Resolution to the Tibet-China Conflict Act',
            'summary': " Promoting a Resolution to the Tibet-China Conflict Act This bill addresses issues relating
to Tibet, including by establishing a statutory definition of Tibet that includes areas in Chinese provinces 
outside the Tibet Autonomous Region (TAR).  For the purposes of U.S. policies and activities relating to Tibet, 
this bill defines Tibet to include the TAR and the Tibetan areas of the Qinghai, Sichuan, Gansu, and Yunnan 
provinces. (Generally, when China's government refers to Tibet, it means only the TAR, while Tibetan exile groups 
consider historical Tibet to include the TAR as well as areas in the provinces included in this bill's definition. 
China's government formally established the TAR in 1965.) Furthermore, the objectives of the Office of the U.S. 
Special Coordinator for Tibetan Issues shall include working to ensure that U.S. government statements and 
documents counter disinformation about Tibet by China's government and the Chinese Communist Party, including 
disinformation about Tibet's history and institutions. The bill also authorizes the office to take other actions to
counter such disinformation.  This bill also states that it is U.S. policy that the conflict between Tibet and 
China is unresolved and that Tibet's legal status remains to be determined in accordance with international law. ",
            'sponsor': 'M000312',
            'source': 'https://www.congress.gov/bill/118th-congress/house-bill/533'
        }
    ),
    Document(
        page_content="Congress finds the following:\n            (1) It has been the long-standing policy of the 
United \n        States to encourage meaningful and direct dialogue between \n        People's Republic of China 
authorities and the Dalai Lama or \n        his representatives, without preconditions, to seek a \n        
settlement that resolves differences.\n            (2) Ten rounds of dialogue held between 2002 and 2010 \n        
between the People's Republic of China authorities and the 14th",
        metadata={
            'key': '118S138',
            'congress_num': '118',
            'legis_class': 'S',
            'legis_num': '138',
            'name': 'Promoting a Resolution to the Tibet-China Conflict Act',
            'summary': " Promoting a Resolution to the Tibet-China Conflict Act This bill addresses issues relating
to Tibet, including by establishing a statutory definition of Tibet that includes areas in Chinese provinces 
outside the Tibet Autonomous Region (TAR).  For the purposes of U.S. policies and activities relating to Tibet, 
this bill defines Tibet to include the TAR and the Tibetan areas of the Qinghai, Sichuan, Gansu, and Yunnan 
provinces. (Generally, when China's government refers to Tibet, it means only the TAR, while Tibetan exile groups 
consider historical Tibet to include the TAR as well as areas in the provinces included in this bill's definition. 
China's government formally established the TAR in 1965.) Furthermore, the objectives of the Office of the U.S. 
Special Coordinator for Tibetan Issues shall include working to ensure that U.S. government statements and 
documents counter disinformation about Tibet by China's government and the Chinese Commu

In [52]:
response = qa("Which fields of industry receive significant federal funding through grants?")
print(response['result'])
print(response['source_documents'])

academic hospitals, and other Federal and non-Federal health entities

[
    Document(
        page_content='2022 of this title);\n                    ``(B) financial management;\n                    
``(C) counseling for addiction issues;\n                    ``(D) educational opportunities;\n                    
``(E) anger management; and\n                    ``(F) career and employment resources.\n            ``(4) Any 
other use the Secretary determines appropriate.\n    ``(d) Notice.--The Secretary shall publish a Notice of Funding
\nOpportunity on the internet website Grants.gov, or such successor',
        metadata={
            'key': '118HR3326',
            'congress_num': '118',
            'legis_class': 'HR',
            'legis_num': '3326',
            'name': 'HAVENS Act',
            'summary': '',
            'sponsor': 'K000381',
            'source': 'https://www.congress.gov/bill/118th-congress/house-bill/3326'
        }
    ),
    Document(
        page_content='National Institutes of Health, the Secretary of Defense, and \n        private sector 
entities.',
        metadata={
            'key': '118HR2854',
            'congress_num': '118',
            'legis_class': 'HR',
            'legis_num': '2854',
            'name': 'To require the Administrator of the United States Agency for International Development to 
submit a report on progress relating to malaria reduction, and for other purposes.',
            'summary': '',
            'sponsor': 'S000522',
            'source': 'https://www.congress.gov/bill/118th-congress/house-bill/2854'
        }
    ),
    Document(
        page_content='academic hospitals, and other Federal and non-Federal health \n        entities.\n           
<all>',
        metadata={
            'key': '118S1125',
            'congress_num': '118',
            'legis_class': 'S',
            'legis_num': '1125',
            'name': 'EHR Program RESET Act of 2023',
            'summary': '',
            'sponsor': 'T000464',
            'source': 'https://www.congress.gov/bill/118th-congress/senate-bill/1125'
        }
    ),
    Document(
        page_content='appropriated for grants under this section $750,000,000 for each of \nfiscal years 2024 
through 2028.\n                                 <all>',
        metadata={
            'key': '118HR3117',
            'congress_num': '118',
            'legis_class': 'HR',
            'legis_num': '3117',
            'name': 'Hotel to Housing Conversion Act of 2023',
            'summary': '',
            'sponsor': 'S001150',
            'source': 'https://www.congress.gov/bill/118th-congress/house-bill/3117'
        }
    ),
    Document(
        page_content="``(2) the National Laboratories; and\n            ``(3) the private sector.\n    ``(e) 
Authorization of Appropriations.--There are authorized to be \nappropriated to carry out this section--\n          
``(1) $30,000,000 for fiscal year 2023;\n            ``(2) $31,250,000 for fiscal year 2024;\n            ``(3) 
$32,500,000 for fiscal year 2025;\n            ``(4) $33,750,000 for fiscal year 2026; and\n            ``(5) 
$35,000,000 for fiscal year 2027.''.",
        metadata={
            'key': '118S429',
            'congress_num': '118',
            'legis_class': 'S',
            'legis_num': '429',
            'name': 'Abandoned Well Remediation Research and Development Act',
            'summary': ' Abandoned Well Remediation Research and Development Act This bill requires the Department 
of Energy (DOE) to establish a research, development, and demonstration program concerning abandoned oil and gas 
wells. Under the program, DOE must work to improve (1) data collection on the location of abandoned oil or gas 
wells; (2) the plugging, remediation, reclamation, and repurposing of the wells; and (3) strategies to mitigate 
potential environmental impacts of documented and undocumented abandoned wells.',
            'sponsor': 'L000570',
            'source': 'https://www.congress.gov/bill/118th-congress/senate-bill/429'
        }
 

In [53]:
response = qa("Describe the crimes of war Russia has carried on in Ukraine.")
print(response['result'])
print(response['source_documents'])

rape, summary execution, and unlawful violence and threats against civilians

[
    Document(
        page_content='Whereas, on February 24, 2022, the Russian Federation invaded the sovereign \n        State 
of Ukraine in violation of international law;\nWhereas Russian troops have since committed horrendous atrocities 
and human \n        rights violations;\nWhereas, as of March 2023, the Russian Government has kidnapped and removed
tens \n        of thousands of innocent children from their homes in Ukraine;\nWhereas the Russian Government 
continues to falsely claim that the parents and',
        metadata={
            'key': '118HCONRES29',
            'congress_num': '118',
            'legis_class': 'HCONRES',
            'legis_num': '29',
            'name': "Condemning the Russian Federation's kidnapping of Ukrainian children.",
            'summary': '',
            'sponsor': 'B001298',
            'source': 'https://www.congress.gov/bill/118th-congress/house-concurrent-resolution/29'
        }
    ),
    Document(
        page_content='Whereas, on February 24, 2022, Russia unleashed a full-scale invasion of Ukraine \n        in
contravention of international law and the Charter of the United \n        Nations;\nWhereas the Russian Armed 
Forces committed mass atrocities in Bucha, Irpin, \n        Kherson, Izyum, Mariupol, Dnipro, and Ukrainian towns 
occupied by the \n        Russian Federation, including rape, summary execution, and unlawful \n        violence 
and threats against civilians;',
        metadata={
            'key': '118SRES227',
            'congress_num': '118',
            'legis_class': 'SRES',
            'legis_num': '227',
            'name': "A resolution calling on the President to support the creation of an international special 
tribunal to prosecute Russia's aggression against Ukraine.",
            'summary': '',
            'sponsor': 'C000141',
            'source': 'https://www.congress.gov/bill/118th-congress/senate-resolution/227'
        }
    ),
    Document(
        page_content='Whereas, on February 24, 2022, the Russian Federation invaded the sovereign \n        State 
of Ukraine in violation of international law;\nWhereas Russian troops have since committed horrendous atrocities 
and human \n        rights violations;\nWhereas, as of March 2023, research indicates that agents of the Government
of \n        the Russian Federation have kidnapped and removed at least 6,000 \n        innocent children from 
their homes in Ukraine, and have likely kidnapped',
        metadata={
            'key': '118SRES128',
            'congress_num': '118',
            'legis_class': 'SRES',
            'legis_num': '128',
            'name': "A resolution condemning the Russian Federation's kidnapping of Ukrainian children.",
            'summary': '',
            'sponsor': 'K000367',
            'source': 'https://www.congress.gov/bill/118th-congress/senate-resolution/128'
        }
    ),
    Document(
        page_content="Whereas the Russian Federation's military tactics and calculations have led to \n        
outright failure in Ukraine;\nWhereas an immense amount of compelling evidence has already been collected \n       
indicating that members of the Russian military have committed a wide \n        variety of atrocity crimes in 
Ukraine and against the Ukrainian people;\nWhereas efforts to achieve justice and accountability for atrocity 
crimes \n        committed by members of Russia's forces in Ukraine have been organized",
        metadata={
            'key': '118HRES41',
            'congress_num': '118',
            'legis_class': 'HRES',
            'legis_num': '41',
            'name': "Underscoring continued support for the people of Ukraine in their fight against Russia's 
aggression, reiterating support for Ukraine's independence, sovereignty, and territorial integrity within its 
internationally recognized borders and its territorial waters, and calling for justice for all atrocity crimes 
committed by members of Russia's forces in Ukraine.",
   

In [54]:
response = qa("How are cryptocurrencies handled differently from normal banking procedures?")
print(response['result'])
print(response['source_documents'])

No Federal reserve bank may hold digital currencies minted or issued by the United States Government as assets or 
liabilities on a balance sheet of the bank

[
    Document(
        page_content='currency directly to an individual (including central bank \n        digital currency issued 
to an individual through a custodial \n        intermediary) or a digital currency intermediary, offer related \n  
products or services directly to an individual, or maintain an \n        account on behalf of an individual 
(including an account in a \n        specially designated account at a digital currency intermediary \n        or 
supervised commercial bank). No Federal reserve bank may',
        metadata={
            'key': '118S967',
            'congress_num': '118',
            'legis_class': 'S',
            'legis_num': '967',
            'name': 'No CBDC Act',
            'summary': '',
            'sponsor': 'L000577',
            'source': 'https://www.congress.gov/bill/118th-congress/senate-bill/967'
        }
    ),
    Document(
        page_content='SEC. 4. CENTRAL BANK DIGITAL CURRENCY STUDY AND PILOT PROGRAM \n              COORDINATION.',
        metadata={
            'key': '118HR1122',
            'congress_num': '118',
            'legis_class': 'HR',
            'legis_num': '1122',
            'name': 'CBDC Anti-Surveillance State Act',
            'summary': ' CBDC Anti-Surveillance State Act  This bill limits the ability of the Federal Reserve to 
(1) provide direct services to individuals, and (2) use a central bank digital currency. A central bank digital 
currency is a digital currency (e.g., Bitcoin or Ether) issued by a government-backed central bank. Specifically, 
the bill prohibits the Federal Reserve and the Federal Open Market Committee from using any central bank digital 
currency to implement monetary policy. In addition, a Federal Reserve bank is prohibited from offering products or 
services directly to an individual, maintaining an account on behalf of an individual, or issuing a central bank 
digital currency directly to an individual.  The Federal Reserve must (1) consult with each Federal Reserve bank 
with respect to any central bank digital currency study or pilot program, and (2) issue quarterly reports on the 
findings and determinations of any such study or program.',
            'sponsor': 'E000294',
            'source': 'https://www.congress.gov/bill/118th-congress/house-bill/1122'
        }
    ),
    Document(
        page_content="or supervised commercial bank). No Federal reserve bank may \n        hold digital currencies
minted or issued by the United States \n        Government as assets or liabilities on a balance sheet of the \n   
bank or use such digital currencies as part of fulfilling the \n        requirements under section 2A.''.\n        
<all>",
        metadata={
            'key': '118S967',
            'congress_num': '118',
            'legis_class': 'S',
            'legis_num': '967',
            'name': 'No CBDC Act',
            'summary': '',
            'sponsor': 'L000577',
            'source': 'https://www.congress.gov/bill/118th-congress/senate-bill/967'
        }
    ),
    Document(
        page_content="securities.'' and inserting ``other forms of securities and in \n        cryptocurrency or 
other digital assets that are issued or \n        transferred using distributed ledger or blockchain technology \n 
including: virtual currencies, coins and tokens or any other \n        digital asset specified by regulations of a 
filer's supervising \n        ethics office.''.\n    (f) Mandatory Electronic Filing.--Section 103(a) of the Ethics
in",
        metadata={
            'key': '118HR389',
            'congress_num': '118',
            'legis_class': 'HR',
            'legis_num': '389',
            'name': 'PORTFOLIO Act',
            'summary': ' Preventing Opportunistic Returns on Trades and Futures by Officials, Leadership, and 
Individuals in Office Act or the PORTFOLIO Act This bill generally prohibits federal employees and officials from 
owning or trading in synthetic assets (i.e., tokenized derivativ